# Monitoring with No Model [Basic]
For this section, we will cover how to begin monitoring with only a prediction
log. We will do so by sending monitoring events for a specific model and 
project we configure.

Specifically, we will:
- Initialize a connection with Fiddler
- Load a prediction log
- Create a project
- Create a model using the prediction log
- Send monitoring events

## Initialize Fiddler Client
We begin this section as usual by establishing a connection to our
Fiddler instance. We can establish this connection either by specifying 
our credentials directly, or by utilizing our `fiddler.ini` file. More
information can be found in the [setup](https://github.com/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/00%20Setup.ipynb) section.

In [ ]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

## Load Prediction Log

Prediction logs must contains the model's input features and predictions. This is information
that can be collected while a model is running in production. For this demonstration, we have
collected events and saved them in a file called `p2p_production.log`.

In [ ]:
import pandas as pd
prediction_log_df = pd.read_csv('/app/fiddler_samples/samples/datasets/p2p_loans/p2p_production.log')
prediction_log_df.head(3)

## Create Project
Projects are one of the key entities of Fiddler. They are convenient containers 
for housing the models and datasets associated with a given ML use case. Specifics about
projects can be found [here](https://docs.fiddler.ai/components/#project)

In [ ]:
project_id = 'tutorial'
model_id = 'no_model_artifact'

if project_id in client.list_projects():
    client.delete_model(project_id, model_id)
    client.delete_dataset(model_id)
else:
    client.create_project(project_id)

## Create Model Using Prediction Log
With our project now created, we will now create a model using the prediction log.
Models are another one of the key entities of Fiddler, and we will create 
this model will be used for generating relevant schema information. More 
information about models can be found [here](https://docs.fiddler.ai/components/#model)

In [ ]:
outputs = ['probability_fully_paid']

is_feature = lambda x: x not in outputs
features = list(filter(is_feature, list(prediction_log_df.columns)))

client.create_model_from_prediction_log(
    project_id,
    model_id,
    prediction_log_df,
    features,
    outputs,
    model_task=fdl.ModelTask.BINARY_CLASSIFICATION,
)

## Send Monitoring Events
In this step, we will be simulating traffic to send for our model monitoring by using 
[publish_event](https://docs.fiddler.ai/api-reference/python-package/#publish-event). 
This will be the equivalent of running our model separately on data, and either 
sending to Fiddler then, or saving this information to a log and sending at a later point.

For this demonstration, we will be going with a log-related approach. 
This log contains rows that have inputs and predictions. 
To most accurately simulate this as a time-series event, we will also be calling 
a function to generate a timestamp in the last two weeks. Real data will ideally 
have a timestamp related to when the event took place; otherwise, the current 
time will be used.

**Note**: The timestamp must be in UTC milliseconds. See 
[here](https://docs.fiddler.ai/api-reference/python-package/#publish-event) for more details

In [ ]:
import datetime
import time
from IPython.display import clear_output
from random import sample, randint
NUM_EVENTS_TO_SEND = 50

def getTimestampFromPastTwoWeeks():
    """
    Generate a randomized timestamp from the past two weeks. Timestamp is in 
    milliseconds since epoch in UTC.
    """
    TWO_WEEKS_MS = 604800 * 2 * 1000
    current_time_in_ms = round(time.time() * 1000)
    
    random_time_in_past_two_weeks = current_time_in_ms - randint(0, TWO_WEEKS_MS)
    return random_time_in_past_two_weeks

# Convert this dataframe into a list of dictionary events, where each event is its own dictionary
event_list_dict = prediction_log_df.sample(n=NUM_EVENTS_TO_SEND, random_state=42).to_dict(orient='records') 

for ind, event_dict in enumerate(event_list_dict):
    event_ms_time_stamp = getTimestampFromPastTwoWeeks()
    result = client.publish_event(project_id, model_id, event_dict, event_time_stamp=event_ms_time_stamp)
    
    clear_output(wait = True)
    readable_timestamp = datetime.datetime.fromtimestamp(event_ms_time_stamp/1000.0)
    
    print(f'Sending {ind+1} / {NUM_EVENTS_TO_SEND} \n{readable_timestamp} UTC: \n{event_dict}')
    time.sleep(1)